In [ ]:
# default_exp data.features

# Featurizing Time Series

> Functions used to transform time series into a dataframe that can be used to create tabular dataloaders.

In [ ]:
#export
from tsai.imports import *
from tsai.utils import *
from tsai.data.external import *
from tsai.data.core import *
from tsai.data.tabular import *

In this case we are using tsfresh that is one of the most widely known libraries used to create features from time series. You can get more details about this library here: https://tsfresh.readthedocs.io/en/latest/

In [ ]:
#export
import tsfresh
from tsfresh import extract_features
from tsfresh.feature_extraction.settings import ComprehensiveFCParameters, MinimalFCParameters, EfficientFCParameters

In [ ]:
#export
def get_ts_features(X:Union[np.ndarray, torch.Tensor], y:Union[None, np.ndarray, torch.Tensor]=None, features:Union[str, dict]='min', 
                    n_jobs:Optional[int]=None, **kwargs):
    """
    Args:
        X: np.array or torch.Tesnor of shape [samples, dimensions, timesteps].
        y: Not required for unlabeled data. Otherwise, you need to pass it.
        features: 'min', 'efficient', 'all', or a dictionary. Be aware that 'efficient' and 'all' may required substantial memory and time.
    """    
    df = to_tsfresh_df(X)
    n_jobs = ifnone(n_jobs, defaults.cpus)
    if 'default_fc_parameters' in kwargs.keys(): default_fc_parameters = default_fc_parameters 
    elif features == 'min': default_fc_parameters = MinimalFCParameters()
    elif features == 'efficient': default_fc_parameters = EfficientFCParameters()
    elif features == 'all': default_fc_parameters = ComprehensiveFCParameters()
    else: default_fc_parameters = None
    df = tsfresh.extract_features(df, column_id="id", n_jobs=n_jobs, default_fc_parameters=default_fc_parameters, **kwargs)
    if y is not None:
        if y.ndim == 1: y = y.reshape(-1,1)
        for i in range(y.shape[-1]):
            df['target' if y.shape[-1] == 1 else f'target_{i}'] = y[:, i]
    return df

In [ ]:
dsid = 'NATOPS'
X, y, splits = get_UCR_data(dsid, return_split=False)
X.shape

(360, 24, 51)

There are 3 levels of fatures you can extract: 'min', 'efficient' and 'all'. I'd encourage you to start with min as feature creation may take a long time. 

In addition to this, you can pass a dictionary to build the desired features (see tsfresh documentation in the link above).

In [ ]:
ts_features_df = get_ts_features(X, y)
ts_features_df.shape

Feature Extraction: 100%|████████████████████████████████████| 40/40 [00:05<00:00,  7.11it/s]


(360, 217)

The 'min' set creates a dataframe with 8 features per channel + 1 per target (total 193) for each time series sample (360).

In [ ]:
cont_names = ts_features_df.columns[:-1]
y_names = 'target'
dls = get_tabular_dls(ts_features_df, splits=splits, cont_names=cont_names, y_names=y_names)
dls.show_batch()

,0__sum_values,0__median,0__mean,0__length,0__standard_deviation,0__variance,0__root_mean_square,0__maximum,0__minimum,1__sum_values,1__median,1__mean,1__length,1__standard_deviation,1__variance,1__root_mean_square,1__maximum,1__minimum,2__sum_values,2__median,2__mean,2__length,2__standard_deviation,2__variance,2__root_mean_square,2__maximum,2__minimum,3__sum_values,3__median,3__mean,3__length,3__standard_deviation,3__variance,3__root_mean_square,3__maximum,3__minimum,4__sum_values,4__median,4__mean,4__length,4__standard_deviation,4__variance,4__root_mean_square,4__maximum,4__minimum,5__sum_values,5__median,5__mean,5__length,5__standard_deviation,5__variance,5__root_mean_square,5__maximum,5__minimum,6__sum_values,6__median,6__mean,6__length,6__standard_deviation,6__variance,6__root_mean_square,6__maximum,6__minimum,7__sum_values,7__median,7__mean,7__length,7__standard_deviation,7__variance,7__root_mean_square,7__maximum,7__minimum,8__sum_values,8__median,8__mean,8__length,8__standard_deviation,8__variance,8__root_mean_square,8__maximum,8__minimum,9__sum_values,9__median,9__mean,9__length,9__standard_deviation,9__variance,9__root_mean_square,9__maximum,9__minimum,10__sum_values,10__median,10__mean,10__length,10__standard_deviation,10__variance,10__root_mean_square,10__maximum,10__minimum,11__sum_values,11__median,11__mean,11__length,11__standard_deviation,11__variance,11__root_mean_square,11__maximum,11__minimum,12__sum_values,12__median,12__mean,12__length,12__standard_deviation,12__variance,12__root_mean_square,12__maximum,12__minimum,13__sum_values,13__median,13__mean,13__length,13__standard_deviation,13__variance,13__root_mean_square,13__maximum,13__minimum,14__sum_values,14__median,14__mean,14__length,14__standard_deviation,14__variance,14__root_mean_square,14__maximum,14__minimum,15__sum_values,15__median,15__mean,15__length,15__standard_deviation,15__variance,15__root_mean_square,15__maximum,15__minimum,16__sum_values,16__median,16__mean,16__length,16__standard_deviation,16__variance,16__root_mean_square,16__maximum,16__minimum,17__sum_values,17__median,17__mean,17__length,17__standard_deviation,17__variance,17__root_mean_square,17__maximum,17__minimum,18__sum_values,18__median,18__mean,18__length,18__standard_deviation,18__variance,18__root_mean_square,18__maximum,18__minimum,19__sum_values,19__median,19__mean,19__length,19__standard_deviation,19__variance,19__root_mean_square,19__maximum,19__minimum,20__sum_values,20__median,20__mean,20__length,20__standard_deviation,20__variance,20__root_mean_square,20__maximum,20__minimum,21__sum_values,21__median,21__mean,21__length,21__standard_deviation,21__variance,21__root_mean_square,21__maximum,21__minimum,22__sum_values,22__median,22__mean,22__length,22__standard_deviation,22__variance,22__root_mean_square,22__maximum,22__minimum,23__sum_values,23__median,23__mean,23__length,23__standard_deviation,23__variance,23__root_mean_square,23__maximum,23__minimum,target
0,-37.142456,-0.566632,-0.728283,51.0,1.080438,1.167346,1.302975,0.696339,-2.332613,-35.411819,-0.415532,-0.694349,51.0,0.832850,0.693640,1.084325,0.291253,-1.906993,-35.485306,-0.702877,-0.695790,51.0,0.298104,0.088866,0.756961,-0.247774,-1.539848,39.676464,0.573882,0.777970,51.0,1.060847,1.125397,1.315536,2.243961,-0.855261,-41.451077,-0.508144,-0.812766,51.0,0.747256,0.558391,1.104074,0.243118,-1.956784,-19.976217,-0.453625,-0.391691,51.0,0.559977,0.313574,0.683370,0.523402,-1.635486,-37.629429,-0.681118,-0.737832,51.0,0.255562,0.065312,0.780838,-0.348886,-1.181166,-24.176161,-0.583717,-0.474042,51.0,0.296457,0.087887,0.559109,0.081907,-0.771776,-13.816839,-0.244598,-0.270918,51.0,0.166769,0.027812,0.318133,-0.016374,-0.672385,37.760784,0.651040,0.740408,51.0,0.263288,0.069320,0.785827,1.151987,0.373290,-28.283892,-0.656930,-0.554586,51.0,0.266712,0.071135,0.615387,-0.046888,-0.849735,-6.133113,-0.115276,-0.120257,51.0,0.273553,0.074831,0.298819,0.297453,-0.711572,-37.277363,-0.612094,-0.730929,51.0,0.724792,0.52532

In [ ]:
x_cat, x_cont, yb = first(dls.train)
x_cont[:10]

tensor([[ 0.8806,  0.1002,  0.8806,  ...,  1.1443, -0.8758, -0.4822],
        [-2.0681, -2.0730, -2.0681,  ...,  0.1335, -0.6172, -0.6517],
        [ 0.5705,  0.9112,  0.5705,  ...,  0.1432, -1.0031,  0.4127],
        ...,
        [-0.5616, -0.7755, -0.5616,  ...,  0.8445, -1.0365, -0.7131],
        [ 0.2748,  0.5201,  0.2748,  ..., -0.2569, -0.4253,  0.2949],
        [-0.1811,  0.0787, -0.1811,  ..., -0.1807,  0.0275,  0.2089]])

In [ ]:
from tsai.models.utils import *
from tsai.models.TabModel import *
model = build_tabular_model(TabModel, dls=dls)
learn = Learner(dls, model, metrics=[accuracy, RocAuc()])
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,time
0,1.861978,1.789068,0.266667,0.535370,00:00
1,1.778144,1.679453,0.477778,0.875778,00:00
2,1.637727,1.548165,0.622222,0.923259,00:00
3,1.509121,1.444736,0.672222,0.933259,00:00
4,1.409377,1.394630,0.650000,0.937111,00:00


In [ ]:
b = first(dls.train)
model(*b[:-1]).shape

torch.Size([64, 6])

In [ ]:
#hide
from tsai.imports import create_scripts
from tsai.export import get_nb_name
nb_name = get_nb_name()
create_scripts(nb_name);

020_data.features.ipynb saved at 2021-10-02 11:56:48.
Converted 020_data.features.ipynb.


Correct conversion! 😃
Total time elapsed 0.106 s
Saturday 02/10/21 11:56:51 CEST
